In [ ]:
!wget -qO- https://raw.githubusercontent.com/nansencenter/DA-tutorials/master/resources/colab_bootstrap.sh | bash -s
from resources.workspace import *
%matplotlib inline

The truth (or "true state", or "nature"), $x_k$, "evolves in time (indexed by $k$)" according to some linear "dynamical" model:
$$ x_{k} = \mathscr{M}_{k-1} x_{k-1} + q_{k-1} \, , \qquad (\text{Dyn}) \, .$$
where $q_k$ is a random noise (process) that accounts for model errors.

Here, $\mathscr{M}_{k-1}$ is just a number. In later tutorials it will be generalized to matrices, and eventually nonlinear operators.

# The (univariate) Kalman filter (KF)

The KF recursively estimates $x_k$ in successive cycles which consist of two steps :

####  Exc 3.1: The forecast step: 
It is supposed that $\;\;\;\quad x_{k-1} \sim \mathcal{N}(\hat{x}_{k-1}, P_{k-1})$,  
and that (independently) $q_{k-1} \sim \mathcal{N}(0, Q)$.  

Show that the mean and variance (i.e. the expected value and the variance) of $x_k$, respectively, are given by:  
$b_k = \mathscr{M}_{k-1} \hat{x}_{k-1}$,     
$B_k = \mathscr{M}_{k-1}^2 P_{k-1} + Q$.

In [ ]:
#show_answer('RV sums')

It can be shown that [the sum of two Gaussian random variables](https://en.wikipedia.org/wiki/Sum_of_normally_distributed_random_variables#Proof_using_convolutions)  is again a Gaussian.  
Thus, the forecast step just requires computing the new moments (mean and variance).

#### The analysis step
"updates" the prior (forecast), $\mathcal{N}(x_k \mid \; b_k,\; B_k)$,  
based on the likelihood, $\quad\;\;\;\, \mathcal{N}(y_k \mid \, x_k, \; R)$,  
into the posterior (analysis), $\; \; \, \mathcal{N}(x_k \mid \; \hat{x}_{k}, \, P_{k})$.  
The update formulae was derived as the Gaussian-Gaussian Bayes' rule in [the previous tutorial](T2%20-%20Bayesian%20inference.ipynb#Gaussian-Gaussian-Bayes).

This completes the cycle, which can then restart with the forecast from $k$ to $k+1$.

# A straight-line example

Many mathematical methods are tagged as "least squares". They typically have one thing in common: some sum of squared terms is being minimized. Both (least-squares) linear regression and Kalman filtering (KF) may be derived from "least squares".
Do they yield the same estimate (when applied to the linear regression problem)? That's what we'll investigate...

Consider the straight line
$$x_k = a k \, ,     \qquad \qquad (1) $$
and suppose we have observations ($y$) of the line, but corrupted by noise ($r$):
\begin{align*}
y_k &= x_k + r_k \, , \;\; \qquad (2)
\end{align*}
where $r_k \sim \mathcal{N}(0, R)$ for some $R>0$.

The code below sets up an experiment based on eqns. (1) and (2).  
Parameters:

In [ ]:
a = 0.4  # Slope (xx[k] = a*k) paramterer. Unknown to be estimated.
K = 10   # Length of experiment (final time index)
R = 1    # Observation noise strength

The following simulates a series of the truth ($x$) and observations ($y$).

In [ ]:
# Recall the naming convention: xx and yy hold time series of x and y.
xx = np.zeros(K+1) # truth states
yy = np.zeros(K+1) # obs

for k in 1+arange(K):
    xx[k] = a*k
    yy[k] = xx[k] + sqrt(R)*randn()

# The obs at k==0 should not be used (since we know xx[0]==0, it is worthless).
yy[0] = np.nan

### Esitmation by linear regression
(Least-squares) linear regression minimizes
$$J_K(\hat{a}) = \sum_{k=1}^K (y_k - \hat{a} k)^2 \, ,  \qquad (4)$$
yielding the estimator
$$\hat{a} = \frac{\sum_{k=1}^K {k} y_{k}}{\sum_{k=1}^K {k}^2} \, . \qquad \qquad (6)$$

**Exc 3.2:** Derive (6) from (4).

In [ ]:
#show_answer('LinReg deriv')

**Exc 3.4:** Program the linear regresson estimator (6)

In [ ]:
def lin_reg(k):
    "Liner regression estimator based on observations y_1, ..., y_k."
    ### INSERT ANSWER HERE ###
    return a

In [ ]:
#show_answer('LinReg_k')

Let's visualize the experiment:

In [ ]:
@interact(k=IntSlider(min=1, max=K))
def plot_experiment(k):
    plt.figure(figsize=(10,6))
    kk = arange(k+1)
    plt.plot(kk,xx[kk]        ,'k' ,label='true state ($x$)')
    plt.plot(kk,yy[kk]        ,'k*',label='noisy obs ($y$)')
    plt.plot(kk,kk*lin_reg(k) ,'r' ,label='Linear regress.')

    ### Uncomment this block AFTER doing the Exc 3.8 ###
    # pw_bb, pw_xxhat = weave_fa(bb,xxhat)
    # pw_kf, pw_ka    = weave_fa(arange(K+1))    
    # plt.plot(pw_kf[:3*k],pw_bb[:3*k]     ,'c'  ,label='KF forecasts')
    # plt.plot(pw_ka[:3*k],pw_xxhat[:3*k]  ,'b'  ,label='KF analyses')
    # #plt.plot(kk,kk*xxhat[k]/k           ,'g--',label='KF extrapolated')

    plt.xlim([0,1.01*K])
    plt.ylim([-1,1.2*a*K])
    plt.xlabel('time index (k)')
    plt.ylabel('$x$, $y$, and $\hat{x}$')
    plt.legend(loc='upper left')
    plt.show()

---
### Estimation by the KF
In the following we tacke the same problem, but using the KF.

The observations equation (2)
yields the likelihood $p(y_k|x_k) = \mathcal{N}(y_k \mid x_k,R)$.  
Hopefully this is intuitive; otherwise, a more detailed derivation will be given in tutorial 4.

**Exc 3.6:** For the (univariate) KF,
we need to reformulate the problem of estimating the parameter $a$ 
as the problem of estimating $x_k$ (yielding $\hat{a}_k = \hat{x}_k / k$).
Derive and implement the "forecast model" $\mathscr{M}_k$ (a function of $k>0$ only) such that
the recursion $x_{k+1} = \mathscr{M}_k x_k$ is equivalent to eqn (1).

In [ ]:
def Mod(k):
    return ### INSERT ANSWER HERE ###

In [ ]:
#show_answer('Sequential 2 Recusive')

**Exc 3.7:** Explain in which sense the KF is optimal for a larger class of problems than linear regression.

In [ ]:
#show_answer('LinReg ⊂ KF')

So although the KF (and its implementation below) may seem like "overkill" for this problem,
this "heavy machinery" can do a lot more, and will pay off later.

**Exc 3.8:** Implement the KF (described at the top of this notebook) in the below code block to estimate $x_k$ for $k=1,\ldots, K$.  

<div class="alert alert-warning" role="alert">
<b>NB:</b> for this example, do not use the "Kalman gain" form of the analysis update.
This problem involves the peculiar, unrealistic situation of infinities
(related to "improper priors") at `k==1`, yielding platform-dependent behaviour.
These peculariaties are of mainly of academic interest, and is not our focus here.
</div>

In [ ]:
Q = 0 # Dynamical model noise strength

# Allocation
bb    = np.zeros(K+1) # mean estimates -- prior/forecast values
xxhat = np.zeros(K+1) # mean estimates -- post./analysis values
BB    = np.zeros(K+1) # var  estimates -- prior/forecast values
PP    = np.zeros(K+1) # var  estimates -- post./analysis values

def KF(k):
    "Cycle k of the Kalman filter"
    # Forecast
    if k==1:
        BB[k] = np.inf # The "initial" prior uncertainty is infinite...
        bb[k] = 0      # ... thus the corresponding mean is inconsequential.
    else:
        BB[k] = ### INSERT ANSWER HERE ###
        bb[k] = ### INSERT ANSWER HERE ###
    # Analysis
    PP[k]    = ### INSERT ANSWER HERE ###
    xxhat[k] = ### INSERT ANSWER HERE ###

# Run estimations/computations
for k in 1+arange(K):
    KF(k)

In [ ]:
#show_answer('KF_k')

**Exc 3.10:** Go back to the animation above and uncomment the block that plots the KF estimates.  
Visually: what is the relationship between the estimates provided by the KF and by linear regression?

In [ ]:
#show_answer('LinReg compare')

<em>Exercises marked with an asterisk (*) are optional.</em>

**Exc 3.12*:** This excercise proves (on paper) the result of the previous excercise.  
Note that the KF forecast step (here with $Q=0$) can be inserted in the analysis step, forming a single couple of recursions, which are recursive:

 * $\hat{x}_k = P_k \big(y_k/R \;+\; \mathscr{M}_{k-1} \hat{x}_{k-1} / [\mathscr{M}_{k-1}^2 P_{k-1}] \big) \qquad (11)$  
 * $P_k = 1/\big(1/R \;+\; 1/[\mathscr{M}_{k-1}^2 P_{k-1}]\big) \qquad \qquad \quad (12)$

Now:

* (a). First show that $P_K = R\frac{K^2}{\sum_{k=1}^K k^2} \, . \;\;\; \qquad \quad (13)$
* (b). Then show that $\hat{x}_K = K\frac{\sum_{k=1}^K k y_k}{\sum_{k=1}^K k^2} = K \hat{a}_K$, where $\hat{a}_K$ is given by eqn (11).

In [ ]:
#show_answer('x_KF == x_LinReg')

#### Exc 3.14:
Let $x_{k+1} = \mathscr{M} x_k$, *for a generic $\mathscr{M}>1$ (note: $Q=0$)*. What does the sequence of $P_k$ converge to?  
You must start from eqn (12), because eqn (13) is for the straight-line example only.

In [ ]:
#show_answer('Asymptotic P when M>1')

#### Exc 3.15:
Redo Exc 3.14, but assuming  
 * (a) $\mathscr{M} = 1$.
 * (b) $\mathscr{M} < 1$.

In [ ]:
#show_answer('Asymptotic P when M=1')
#show_answer('Asymptotic P when M<1')

Thus, if $\mathscr{M}>1$, the KF does not converge to 0 error. This is because, even though you keep gaining more information, this is balanced by the growth in uncertainty by the forecast. On the other hand, if $\mathscr{M} \leq 1$ then the error converges to zero.

In general, however, $\mathscr{M}$ depends on $k$, as will the other system matrices ($Q, R$).
Then, there is no limit value that the state distribution (and its parameters) converges to.

**Exc 3.18*:** Set $Q$ to 1 or more. Re-compute the KF estimates. By interpreting the meaning of $Q$, explain why the KF estimate is now closer to the obs (always at the latest time instance) than the linear regression estimate.

**Exc 3.20*:** Now change $R$ (but don't re-run the simulation of the truth and obs). The KF estimates should not change (in this particular example). Why?

### Summary
The KF consists of two steps:
 * Forecast
 * Analysis
 
In each step, the mean and variance must be updated. 

As an example, we saw that the linear regression estimate is reproduced by the KF, although it is a bit tricky to initialize the KF with infinite uncertainty. However, the KF (i.e. state estimation) is much more general.

### Next: [Multivariate Kalman](T4%20-%20Multivariate%20Kalman.ipynb)